In [52]:
import findspark
findspark.init()

import pyspark
from pyspark import sql

import pandas as pd

In [53]:
spark = SparkSession\
    .builder\
    .appName("MostPopularMovieWithRatingAbove4DataFrames")\
    .getOrCreate()

In [54]:
path_ratings = r"/home/blake/PycharmProjects/BigDataMovies/the-movies-dataset/ratings.csv"
path_movies_metadata = r"/home/blake/PycharmProjects/BigDataMovies/the-movies-dataset/movies_metadata.csv"

ratings = spark.read.csv(
    path=path_ratings,
    header="true",
    inferSchema="true"
)

movies_metadata = spark.read.csv(
    path=path_movies_metadata,
    header="true",
    inferSchema="true",
)

In [55]:
ratings

DataFrame[userId: int, movieId: int, rating: double, timestamp: int]

In [56]:
movies_metadata

DataFrame[adult: string, belongs_to_collection: string, budget: string, genres: string, homepage: string, id: string, imdb_id: string, original_language: string, original_title: string, overview: string, popularity: string, poster_path: string, production_companies: string, production_countries: string, release_date: string, revenue: string, runtime: string, spoken_languages: string, status: string, tagline: string, title: string, video: string, vote_average: string, vote_count: string]

In [57]:
movies_metadata = movies_metadata[['id', 'original_title']]
movies_metadata.take(5)

[Row(id='862', original_title='Toy Story'),
 Row(id='8844', original_title='Jumanji'),
 Row(id='15602', original_title='Grumpier Old Men'),
 Row(id='31357', original_title='Waiting to Exhale'),
 Row(id='11862', original_title='Father of the Bride Part II')]

In [58]:
avg_ratings = ratings\
    .groupBy('movieId')\
    .mean('rating')
avg_ratings.take(5)

[Row(movieId=1645, avg(rating)=3.516589990241182),
 Row(movieId=1591, avg(rating)=2.6416020262782967),
 Row(movieId=3175, avg(rating)=3.586550320670942),
 Row(movieId=1580, avg(rating)=3.5733178489322874),
 Row(movieId=68135, avg(rating)=3.097457627118644)]

In [59]:
count_ratings = ratings\
    .groupBy('movieId')\
    .count()
count_ratings.take(5)

[Row(movieId=1645, count=14346),
 Row(movieId=1591, count=6317),
 Row(movieId=3175, count=16216),
 Row(movieId=1580, count=42193),
 Row(movieId=68135, count=2478)]

In [60]:
count_ratings_above_500 = count_ratings\
    .filter(count_ratings['count'] > 500)
count_ratings_above_500.take(5)

[Row(movieId=1645, count=14346),
 Row(movieId=1591, count=6317),
 Row(movieId=3175, count=16216),
 Row(movieId=1580, count=42193),
 Row(movieId=68135, count=2478)]

In [61]:
avg_ratings_and_counts = avg_ratings.join(
    count_ratings_above_500, on='movieId')

avg_ratings_and_counts.take(5)

[Row(movieId=471, avg(rating)=3.654817548175482, count=12195),
 Row(movieId=833, avg(rating)=2.7146546158812135, count=1549),
 Row(movieId=1088, avg(rating)=3.239810636881426, count=13519),
 Row(movieId=1238, avg(rating)=3.9629796163069546, count=3336),
 Row(movieId=1342, avg(rating)=2.9637979902087093, count=3881)]

In [62]:
movie_titles_avg_ratings_and_counts = avg_ratings_and_counts\
    .join(movies_metadata, 
          avg_ratings_and_counts['movieId']
          == movies_metadata['id'])

movie_titles_avg_ratings_and_counts.take(5)

[Row(movieId=471, avg(rating)=3.654817548175482, count=12195, id='471', original_title='Bandyta'),
 Row(movieId=833, avg(rating)=2.7146546158812135, count=1549, id='833', original_title='Umberto D.'),
 Row(movieId=1088, avg(rating)=3.239810636881426, count=13519, id='1088', original_title='Whale Rider'),
 Row(movieId=1580, avg(rating)=3.5733178489322874, count=42193, id='1580', original_title='Rope'),
 Row(movieId=1591, avg(rating)=2.6416020262782967, count=6317, id='1591', original_title='Nirgendwo in Afrika')]

In [63]:
movie_titles_avg_ratings_and_counts_sorted = \
    movie_titles_avg_ratings_and_counts.orderBy('avg(rating)', ascending=False)
movie_titles_avg_ratings_and_counts_sorted.take(5)

[Row(movieId=318, avg(rating)=4.429014514393623, count=91082, id='318', original_title='The Million Dollar Hotel'),
 Row(movieId=858, avg(rating)=4.339810758717364, count=57070, id='858', original_title='Sleepless in Seattle'),
 Row(movieId=527, avg(rating)=4.266530696698294, count=67662, id='527', original_title='Once Were Warriors'),
 Row(movieId=2019, avg(rating)=4.255073602972702, count=13994, id='2019', original_title='Hard Target'),
 Row(movieId=2959, avg(rating)=4.2307160469145675, count=60024, id='2959', original_title='License to Wed')]

In [64]:
results = movie_titles_avg_ratings_and_counts_sorted[[
    'original_title', 'avg(rating)', 'count'
]]
results.show(truncate=False)

+------------------------------------+------------------+-----+
|original_title                      |avg(rating)       |count|
+------------------------------------+------------------+-----+
|The Million Dollar Hotel            |4.429014514393623 |91082|
|Sleepless in Seattle                |4.339810758717364 |57070|
|Once Were Warriors                  |4.266530696698294 |67662|
|Hard Target                         |4.255073602972702 |13994|
|License to Wed                      |4.2307160469145675|60024|
|The Thomas Crown Affair             |4.2143927037912325|30043|
|Murder She Said                     |4.213030410183875 |28280|
|Cousin, Cousine                     |4.202589307120594 |20855|
|Dead Man                            |4.200819672131147 |7930 |
|三枪拍案惊奇                        |4.199038891372374 |8948 |
|The Good Thief                      |4.187872863087181 |19947|
|Confession of a Child of the Century|4.182070707070707 |39600|
|End of the World                    |4.180264